In [ ]:
!curl -L https://app.roboflow.com/ds/YCm3CoNdC7?key=2EBYxipTM6 > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
!pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [ ]:
# install detectron2:
!pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import torch, torchvision

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()



# import some common detectron2 utilities
from detectron2.data import detection_utils as utils
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import load_coco_json
import detectron2.data.transforms as T

from detectron2.evaluation import COCOEvaluator, inference_on_dataset


In [ ]:
# import some common libraries
import numpy as np
import pandas as pd
import cv2
import os
import random
import copy
from pathlib import Path
import matplotlib.pyplot as plt
import ast
import seaborn as sns

In [ ]:
# --- Read data ---
# TRAIN_PATH = '/kaggle/input/tensorflow-great-barrier-reef'
# Read in the data CSV files
df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
df.head(5)

In [ ]:
# !curl -L https://app.roboflow.com/ds/nLiiHYKSAX?key=89k9yeQp9G > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from detectron2.data.datasets import register_coco_instances
DatasetCatalog.clear()

register_coco_instances("coco_train_dataset", {}, "./train/_annotations.coco.json" ,   "./train")
register_coco_instances("coco_test_dataset", {}, "./valid/_annotations.coco.json" ,   "./valid")


In [ ]:
dataset_train_metadata = MetadataCatalog.get("coco_train_dataset")
dataset_train = DatasetCatalog.get("coco_train_dataset")

In [ ]:
dataset_test_metadata = MetadataCatalog.get("coco_test_dataset")
dataset_test = DatasetCatalog.get("coco_test_dataset")

In [ ]:
import random
from detectron2.utils.visualizer import Visualizer

fig, ax = plt.subplots(3, 1, figsize =(20,20))
i=0

for d in random.sample(dataset_train, 3):
    img = cv2.imread(d["file_name"])
    
    #print(d)
    print("Nombre y ruta del archivo:", d.get('file_name'))
    print("Id de la imagen:", d.get('image_id'))
    print("")
    print("Información de la primera máscara:")
    #print(d['annotations'][0].keys())
    print("Formato del BBox: ",d['annotations'][0].get('bbox_mode'))
    print("Ubicación xy, y dimensiones WH: ",d['annotations'][0].get('bbox'))
    print("Categoría:",d['annotations'][0].get('category_id')+1)
    #print(d['annotations'][0].get('iscrowd'))
    
    visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    
    ax[i].grid(False)
    ax[i].axis('off')
    ax[i].imshow(cv2.cvtColor(vis.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    i += 1


In [ ]:
def custom_mapper (dataset_dict):
    # Implement a mapper, similar to the default DatasetMapper, but with your own customizations
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")

    image, transforms = T.apply_transform_gens([
        T.RandomFlip(prob=0.20, horizontal=True, vertical=False),
        T.RandomFlip(prob=0.20, horizontal=False, vertical=True),
        T.RandomApply(T.RandomBrightness(intensity_min=0.75, intensity_max=1.25),
                      prob=0.20),
        T.RandomApply(T.RandomContrast(intensity_min=0.76, intensity_max=1.25),
                      prob=0.20),
        T.RandomApply(T.RandomCrop(crop_type="relative_range", crop_size=(0.8, 0.8)), 
                      prob=0.20),
        T.RandomApply(T.RandomSaturation(intensity_min=0.75, intensity_max=1.25), 
                      prob=0.20),
#         T.Resize(shape=(1024, 1024)),
        T.RandomApply(T.RandomRotation(angle=[-30,30], expand=True, center=None, sample_style="range", interp=None), 
                      prob=0.20)

    ], image)
    
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg)
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

In [ ]:
cfg = get_cfg() # initialize cfg object
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))  # load default parameters for Mask R-CNN
cfg.DATASETS.TRAIN = ("coco_train_dataset",)  # dataset used for training model
cfg.DATASETS.TEST = ("coco_test_dataset", )  # we will look at the predictions on both sets after training

cfg.SOLVER.CHECKPOINT_PERIOD = 1000  # number of iterations after which to save model checkpoints
cfg.MODEL.DEVICE='cuda'  # 'cpu' to force model to run on cpu, 'cuda' if you have a compatible gpu
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2 # number of images per batch (across all machines)
cfg.TEST.EVAL_PERIOD = 1000
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 8000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512




cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.OUTPUT_DIR = "./"
  

trainer = MyTrainer(cfg)


In [ ]:
train_data_loader = trainer.build_train_loader(cfg)
data_iter = iter(train_data_loader)
batch = next(data_iter)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize =(40,20))

per_image = batch[:1555][0]

img = cv2.imread(per_image["file_name"])



ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[0].set_title("Original image")


img = per_image["image"].permute(1, 2, 0).cpu().detach().numpy()
img = utils.convert_image_to_rgb(img, cfg.INPUT.FORMAT)

visualizer = Visualizer(img, scale=0.5)
target_fields = per_image["instances"].get_fields()
labels = None
vis = visualizer.overlay_instances(
        labels=labels,
        boxes=target_fields.get("gt_boxes", None),
        masks=target_fields.get("gt_masks", None),
        keypoints=target_fields.get("gt_keypoints", None),
    )
ax[1].imshow(cv2.cvtColor(vis.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
ax[1].set_title("Augmentated")

plt.show()

In [ ]:
# Set trainer:
trainer.resume_or_load(resume=True)

# Perform training:
trainer.train()

In [ ]:
cfg = get_cfg() # initialize cfg object
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))  # load default parameters for Mask R-CNN
cfg.MODEL.DEVICE='cuda'  # 'cpu' to force model to run on cpu, 'cuda' if you have a compatible gpu
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.MODEL.WEIGHTS = '../input/model-31-01/model_0003999.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

In [ ]:
random_testing = random.sample(dataset_test, 20)

In [ ]:
for i in range(20):
    print(i)
    fig, ax = plt.subplots(2, 1, figsize =(15,15))
    img = cv2.imread(random_testing[i]['file_name'])

    visualizer = Visualizer(img[:, :, ::-1],
                          metadata=dataset_test_metadata,
                          scale=0.5)
    out = visualizer.draw_dataset_dict(random_testing[i])
    ax[0].imshow(cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    ax[0].set_title("Imagen original")
    outputs = predictor(img)
    visualizer_pred = Visualizer(img[:, :, ::-1],
                              metadata=dataset_test_metadata,
                              scale=0.5)
    pred = visualizer_pred.draw_instance_predictions(outputs['instances'].to('cpu'))
  
    ax[1].imshow(cv2.cvtColor(pred.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    ax[1].set_title("Predicción")
    plt.show()